In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
pd.set_option('display.max_columns',25)

In [ ]:
credit_data = pd.read_csv("C:/Users/I_am_AlexX/Downloads/german.data",sep=' ',names=["Estado_de_la_cuenta_de_cheques_existente","Duración_en_meses","Historial_de_crédito","Propósito","Monto_de_crédito","Cuenta_de_ahorros","Empleo_actual_desde","Tasa_de_cuota_en_porcentaje_del_ingreso_disponible","Estado_personal_y_sexo","Otros_deudores","Residencia_actual_desde","Propiedad","Edad_en_años","Otros_planes_de_cuotas","Alojamiento","Número_de_créditos_existentes_en_este_banco","Trabajo","Número_de_personas_que_están_obligadas_a_proporcionar_alimentos_a","Teléfono","Trabajador_extranjero","Clase"])

In [ ]:
credit_data.head()

In [ ]:
credit_data.shape

In [ ]:
credit_data['Clase'] = credit_data['Clase']-1

In [ ]:
credit_data['Clase'] 

In [ ]:
credit_data.head()

In [ ]:
#credit_data.groupby(["Historial_de_crédito"]).count()

In [ ]:
credit_data.groupby(["Historial_de_crédito"])['Clase'].agg(['count','sum'])

In [ ]:
#credit_data.groupby(['Historial_de_crédito']).sum()

In [ ]:
credit_data.groupby(['Historial_de_crédito']).count()

In [ ]:
#credit_data.groupby(['Historial_de_crédito'])["Clase"].mean()

In [ ]:
#credit_data.groupby(['Historial_de_crédito'])["Clase"].count()

In [ ]:
#credit_data.groupby(['Historial_de_crédito'])["Clase"].sum()

In [ ]:
# El valor de la variable de clase "1" indica incumplimiento y "0" describe no incumplimiento

# Si el tipo de datos es objeto, esto significa que es una variable categórica y cualquier otra variable, como int64, etc., 
# se tratará como continua y se clasificará en 10 partes iguales (también conocidas como deciles) según corresponda.

def IV_calc(data,var):
    if data[var].dtypes == "object":
        
        dataf = data.groupby([var])['Clase'].agg(['count','sum']) #.agg--> Agregue usando una o más operaciones sobre el 
        # eje especificado. 
        # Los datos se agrupan por "var" (la variable en cuestión) mostrando sólo la columna correspondiente a la variable
        # "Clase". Y efectuando las operaciones "count" y "sum" . La primera cuenta el número de filas en el data set original 
        #que contienen alguna de las etiquetas de "Historial_de_crédito"(por ejemplo), estos son: "A30,A31,A32,A33 y A34". En cuanto a sum()
        # suma los valores que tiene cada etiqueta ya sea "0" o "1"
         
        dataf.columns = ["Total","incumplido"]
        dataf["cumplido"] = dataf["Total"] - dataf["incumplido"] # cuenta total de personas cumplidas
        
        dataf["per_incump"] = dataf["incumplido"]/dataf["incumplido"].sum() # porcentaje del total de personas incumplidas 
                                                                            # para cada Historial de crédito específico
        
        dataf["per_cump"] = dataf["cumplido"]/dataf["cumplido"].sum()       # porcentaje del total de personas cumplidas 
                                                                            # para cada Historial de crédito específico
            
        dataf["I_V"] = (dataf["per_cump"] - dataf["per_incump"])*np.log(dataf["per_cump"]/dataf["per_incump"]) # Cálculo
        
        # explícito del valor de la información o "Information Value"
                
        return dataf
    else:
        
        data['bin_var'] = pd.qcut(data[var].rank(method='first'),10) # qcut, Discretiza la variable en cubos de igual tamaño 
                                                                     # según el rango o según los cuantiles de muestra. 
                                                                     # Por ejemplo, 1000 valores para 10 cuantiles producirían 
                                                                     # un objeto categórico que indicaría la pertenencia al 
                                                                     #cuantil para cada punto de datos.
        
        
                                                                     # rank->asigna un número según el valor que tenga en 
                                                                     #la columna del menor(0) al mayor(1000)
            
        dataf = data.groupby(['bin_var'])['Clase'].agg(['count','sum']) #.agg--> Agregue usando una o más operaciones sobre el 
        # eje especificado. 
        # Los datos se agrupan por "bin_var" (es decir por cada uno de los 10 cuantiles) mostrando sólo la columna correspondiente 
        #a la variable "Clase". Y efectuando las operaciones "count" y "sum" . La primera cuenta el número de filas en el data set 
        #original que contienen alguna de las etiquetas de "bin_var", estos son: "(0.999, 100.9],(100.9, 200.8],(300.7, 400.6]
        #,etc". En cuanto a sum() suma los valores que tiene cada etiqueta ya sea "0" o "1"
        
        dataf.columns = ["Total","incumplido"] #asignando headers a las columnas
        
        dataf["cumplido"] = dataf["Total"] - dataf["incumplido"]  # cuenta total de personas cumplidas
        
        dataf["per_incump"] = dataf["incumplido"]/dataf["incumplido"].sum() # porcentaje del total de personas incumplidas 
                                                                            # para cada valor de "Duración en meses" específico
        
        dataf["per_cump"] = dataf["cumplido"]/dataf["cumplido"].sum() # porcentaje del total de personas incumplidas 
                                                                      # para cada valor de "Duración en meses" específico
            
        dataf["I_V"] = (dataf["per_cump"] - dataf["per_incump"])*np.log(dataf["per_cump"]/dataf["per_incump"]) # Cálculo
        
        # explícito del valor de la información o "Information Value"
                
                
        return dataf

## Cálculo del Information Value para 'Historial_de_crédito'

In [ ]:
print(IV_calc(credit_data,'Historial_de_crédito'))

In [ ]:
IV_calc(credit_data,'Historial_de_crédito').I_V.sum()

## Cálculo del Information Value para 'Duración_en_meses'

In [ ]:
print(IV_calc(credit_data,'Duración_en_meses'))

In [ ]:
IV_calc(credit_data,'Duración_en_meses').I_V.sum()

In [ ]:
#pd.qcut(credit_data['Duración_en_meses'].rank(method='first'),10)

In [ ]:
#credit_data['Duración_en_meses'].rank(method='first') 

In [ ]:
#credit_data['Duración_en_meses']

In [ ]:
#for seq in range(5):
#    print(seq)

In [ ]:
#pd.qcut(range(5), 4)

In [ ]:
#credit_data['bin_var'] = pd.qcut(credit_data['Duración_en_meses'].rank(method='first'),10)

credit_data['bin_var'] 

In [ ]:
#credit_data.groupby(['bin_var'])['Clase'].agg(['count','sum'])

## Cálculo del Information Value automatizado 

In [ ]:
# El siguiente módulo, crea un array con las variables que deseen obtener del header
# Data Frame "data"

#Parametros: data , n 

def get_variables(data,n):
    
    header_arr = sorted(data) #obtener un array que contiene los elementos del header del DataFrame
    num_variables_extracted = range(n)
    variables = np.array(header_arr)[num_variables_extracted]
    
    return variables

In [ ]:
var_arr=get_variables(credit_data,21)

In [ ]:
var_arr

In [ ]:
np.delete(var_arr,1)

In [ ]:
def IV_calc_auto(data):
    
    arr=[]
    
    for var in np.delete(var_arr,1):
        
        IV = IV_calc(data,var)
        arr.append(IV)
    return arr

In [ ]:
IV_calc_auto(credit_data)

In [ ]:
arr1=[]
arr2=[]
for dataset in IV_calc_auto(credit_data):
    arr1.append(dataset.I_V.sum())
for var in np.delete(var_arr,1):
      arr2.append(var)
      

In [ ]:
arr2

In [ ]:
np.concatenate((arr1,arr2), axis=0)

In [ ]:
IV_calc_auto(credit_data)[0].I_V.sum()

In [ ]:
IV_calc_auto(credit_data)[1].I_V.sum()

In [ ]:
d = {'Variable': arr2, 'Valor de la información (Information Value)': arr1}
df1 = pd.DataFrame(data=d)

In [ ]:
df1

In [ ]:
df1.sort_values(by='Valor de la información (Information Value)', ascending=False)

# Solución alternativa al problema de automatizar el cálculo de IV (Information Value)

In [ ]:
credit_data.head()

In [ ]:
arr_tot=credit_data.columns.values

In [ ]:
arr_tot

In [ ]:
arr_tot=np.delete(arr_tot,(20,21))

In [ ]:
arr_tot.tolist()

In [ ]:
#discrete_columns = ['Estado_de_la_cuenta_de_cheques_existente','Historial_de_crédito', 'Propósito','Cuenta_de_ahorros', 'Empleo_actual_desde,','Tasa_de_cuota_en_porcentaje_del_ingreso_disponible',
#       'Estado_personal_y_sexo', 'Otros_deudores','Propiedad','Otros_planes_de_cuotas', 'Alojamiento', 'Trabajo','Teléfono', 'Trabajador_extranjero']


#continuous_columns = ['Duración_en_meses', 'Monto_de_crédito','Tasa_de_cuota_en_porcentaje_del_ingreso_disponible',
#'Residencia_actual_desde', 'Edad_en_años','Otros_planes_de_cuotas', 'Alojamiento','Número_de_créditos_existentes_en_este_banco',
#'Número_de_personas_que_están_obligadas_a_proporcionar_alimentos_a']
#discrete_columns + continuous_columns


total_columns = credit_data.columns.values
arr_tot=np.delete(total_columns,(20,21)).tolist()

# List of IV values
Iv_list = []

for col in arr_tot:
    assigned_data = IV_calc(data = credit_data,var = col)
    iv_val = round(assigned_data["I_V"].sum(),3)
    dt_type = credit_data[col].dtypes
    Iv_list.append((iv_val,col,dt_type))

Iv_list = sorted(Iv_list,reverse = True)


for i in range(len(Iv_list)):
    
    print (Iv_list[i][0],",",Iv_list[i][1],",type =",Iv_list[i][2])

In [ ]:
dummy_estche =pd.get_dummies(credit_data['Estado_de_la_cuenta_de_cheques_existente'],
prefix='estado_cuenta_exist')

dummy_hc = pd.get_dummies(credit_data['Historial_de_crédito'],
prefix='hist_cred')

dummy_cueah = pd.get_dummies(credit_data['Cuenta_de_ahorros'],
prefix='cuent_ahorros')

dummy_prop = pd.get_dummies(credit_data['Propósito'],
prefix='propósito')

dummy_property = pd.get_dummies(credit_data['Propiedad'],
prefix='propiedad')

dummy_othinstpln =pd.get_dummies(credit_data['Otros_planes_de_cuotas'],
prefix='otros_planes_cuot')

dummy_empact = pd.get_dummies(credit_data['Empleo_actual_desde'], prefix='emp_act_desde')

dummy_perssx = pd.get_dummies(credit_data['Estado_personal_y_sexo'],
prefix='estado_pers_sexo')

dummy_forgnwrkr = pd.get_dummies(credit_data['Trabajador_extranjero'],
prefix='trabajador_extranj')

dummy_othdts = pd.get_dummies(credit_data['Otros_deudores'],
prefix='otros_deud')

continuous_columns = ['Duration_in_month', 'Credit_amount',
'Installment_rate_in_percentage_of_disposable_income', 'Age_in_years',
'Number_of_existing_credits_at_this_bank']

continuous_columns = ['Duración_en_meses', 'Monto_de_crédito','Tasa_de_cuota_en_porcentaje_del_ingreso_disponible',
                      'Edad_en_años','Número_de_créditos_existentes_en_este_banco']

credit_continuous = credit_data[continuous_columns]

credit_data_new = pd.concat([dummy_estche,dummy_hc,dummy_cueah
,dummy_prop,dummy_property,dummy_othinstpln,dummy_empact
,dummy_perssx,dummy_forgnwrkr,dummy_othdts,credit_continuous,credit_data['Clase']],axis=1)

In [ ]:
credit_data_new.columns.values

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(
credit_data_new.drop(['Clase'] ,axis=1),credit_data_new['Clase'],train_size
= 0.7,random_state=42)
#y_train = pd.DataFrame(y_train)
#y_test = pd.DataFrame(y_test)

In [ ]:
remove_cols_extra_dummy = ['estado_cuenta_exist_A11', 'hist_cred_A30',
'propósito_A40', 'cuent_ahorros_A61','emp_act_desde_A71','estado_pers_sexo_A91',
'otros_deud_A101','propiedad_A121', 'otros_planes_cuot_A141','trabajador_extranj_A201']

In [ ]:
# Aquí, hemos creado la lista adicional para eliminar variables insignificantes paso a paso de forma iterativa mientras 
# trabajamos en la metodología de eliminación hacia atrás; después del final de cada iteración, seguiremos agregando la 
# variable más insignificante y multicolineal a la lista remove_cols_insig, para que esas variables se eliminen mientras 
# se entrena el modelo.
remove_cols_insig = []
remove_cols = list(set(remove_cols_extra_dummy+remove_cols_insig))

In [ ]:
remove_cols

# logistic_model

In [ ]:
import statsmodels.api as sm
logistic_model = sm.Logit(y_train, sm.add_constant(x_train.drop(
remove_cols, axis=1))).fit()
print (logistic_model.summary())

## Cálculo del VIF

In [ ]:
#x_train

In [ ]:
#x_train.columns

In [ ]:
#x_train.drop(remove_cols,axis=1).columns

In [ ]:
cnames = x_train.drop(remove_cols,axis=1).columns #elimina la primera columna de cada categoría en x_train
for i in np.arange(0,len(cnames)): 
    xvars = list(cnames) 
    
yvar = xvars.pop(1)
print(yvar)

In [ ]:
print ("\nFactor de inflación de varianza(Variance Inflation Factor)")
cnames = x_train.drop(remove_cols,axis=1).columns #elimina la primera columna de cada categoría en x_train
for i in np.arange(0,len(cnames)): 
    xvars = list(cnames) 
    yvar = xvars.pop(i)
#print(xvars)
#print(((yvar)))
    print(sm.add_constant(x_train.drop(remove_cols,axis=1)[xvars]))

In [ ]:
x_train.drop(remove_cols,axis=1)

In [ ]:
print ("\nFactor de inflación de varianza(Variance Inflation Factor)")
cnames = x_train.drop(remove_cols,axis=1).columns #elimina la primera columna de cada categoría en x_train


for i in np.arange(0,len(cnames)): # se crea un array Numpy que va desde 0 hasta a longitud de cnames (en este caso 40)
    
    xvars = list(cnames) #lista que contiene los nombres de las columnas ya con el filtro de "cnames" 
    
    yvar = xvars.pop(i) # los 40 valores de cada columna siendo esttas las variables de respuesta
    mod = sm.OLS(x_train.drop(remove_cols,axis=1)[yvar], sm.add_constant(x_train.drop(remove_cols,axis=1)[xvars])) #calcula
    # los mínimos cuadrados ordinarios (OLS)
    # model = sm.OLS(Y,X) <---- sintaxis
    
    # X ---> sm.add_constant(x_train.drop(remove_cols,axis=1)[xvars]) ---> 40 data sets cuya primera columna es una columna de "unos"
    # agregada a cada data set que antes poseía 39 coumnas al sere extraído una columna del data set original
    
    # Y ---> Cada header de cada columna se usa como variable de respuesta
    
    res = mod.fit()
    print(res)

In [ ]:
print ("\nFactor de inflación de varianza(Variance Inflation Factor)")
cnames = x_train.drop(remove_cols,axis=1).columns #elimina la primera columna de cada categoría en x_train

for i in np.arange(0,len(cnames)): # se crea un array Numpy que va desde 0 hasta a longitud de cnames (en este caso 40)
    
    xvars = list(cnames) #40 listas que contiene los nombres de las columnas ya con el filtro de "cnames" (cada una tiene 39)
    
    yvar = xvars.pop(i) # los 40 valores de cada columna siendo esttas las variables de respuesta
    mod = sm.OLS(x_train.drop(remove_cols,axis=1)[yvar], x_train.drop(remove_cols,axis=1)[xvars]) #calcula
    # los mínimos cuadrados ordinarios (OLS)
    # model = sm.OLS(Y,X) <---- sintaxis
    
    # X ---> sm.add_constant(x_train.drop(remove_cols,axis=1)[xvars]) ---> 40 data sets cuya primera columna es una columna de "unos"
    # agregada a cada data set que antes poseía 39 coumnas al sere extraído una columna del data set original
    
    # Y ---> x_train.drop(remove_cols,axis=1)[yvar] ---> Cada header de cada columna se usa como variable de respuesta
    
    res = mod.fit() # Se calculan 40 regresiones con mínimos cyadrados
    vif = 1/(1-res.rsquared)
    print (yvar,round(vif,3))

# Cálculo correcto del VIF

In [ ]:
print ("\nFactor de inflación de varianza(Variance Inflation Factor)")
cnames = x_train.drop(remove_cols,axis=1).columns #elimina la primera columna de cada categoría en x_train reduciendo el data
#set original de 50 columnas a 40 columnas

for i in np.arange(0,len(cnames)): # se crea un array Numpy que va desde 0 hasta a longitud de cnames (en este caso 40)
    
    xvars = list(cnames) #lista que contiene los nombres de las columnas ya con el filtro de "cnames"
    
    yvar = xvars.pop(i) # los 40 valores de cada columna siendo esttas las variables de respuesta (pop(i)) extrae el elemento
    #i-ésimo de la lista xvars
    
    mod = sm.OLS(x_train.drop(remove_cols,axis=1)[yvar], sm.add_constant(x_train.drop(remove_cols,axis=1)[xvars])) #calcula
    # los mínimos cuadrados ordinarios (OLS)
    # model = sm.OLS(Y,X) <---- sintaxis
    
    # X ---> sm.add_constant(x_train.drop(remove_cols,axis=1)[xvars]) ---> 40 data sets cuya primera columna es una columna de 
    # "unos" agregada a cada data set que antes poseía 39 coumnas al serle extraído una columna del data set original
    
    # Se agrega una columna de unos como una variable dependiente. La razón por la que tiene que hacer esto 
    # es porque la función de regresión asume que si no hay una columna constante, entonces desea ejecutar la regresión sin 
    #una intersección. 
    
    # Y ---> x_train.drop(remove_cols,axis=1)[yvar] ---> Cada header de cada columna se usa como variable de respuesta
    
    res = mod.fit() # Se calculan 40 regresiones con mínimos cuadrados
    vif = 1/(1-res.rsquared)
    print (yvar,round(vif,3))

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(dataset):
    vif=pd.DataFrame()
    vif["features"] = dataset.columns 
    vif["VIF_value"]=[variance_inflation_factor(dataset.values,i) for i in range(dataset.shape[1])] 
    
    return(vif)

In [ ]:
calculate_vif(x_train)

In [ ]:
import statsmodels.api as sm
import numpy as np
duncan_prestige = sm.datasets.get_rdataset("Duncan", "carData")
Y = duncan_prestige.data['income']
X = duncan_prestige.data['education']
#X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.params

# Cálculo del estadístico C

In [ ]:
logistic_model.predict(sm.add_constant(x_train.drop(remove_cols,axis=1)))

In [ ]:
sm.add_constant(x_train.drop(remove_cols,axis=1))

In [ ]:
logistic_model.predict()

In [ ]:
len(logistic_model.predict())

In [ ]:
sm.add_constant(x_train.drop(remove_cols,axis=1))

In [ ]:
logistic_model.predict(sm.add_constant(x_train.drop(remove_cols,axis=1)))

In [ ]:
logistic_model

In [ ]:
# import statsmodels.api as sm
# logistic_model = sm.Logit(y_train, sm.add_constant(x_train.drop(
# remove_cols, axis=1))).fit()
# print (logistic_model.summary())

In [ ]:
logistic_model.predict(sm.add_constant(x_train.drop (remove_cols,axis=1)))

In [ ]:
y_pred = pd.DataFrame(logistic_model.predict(sm.add_constant(x_train.drop (remove_cols,axis=1)))) #Usamos e conjunto de prueba 
# para prededcir el valor de la variable de respuesta o a predecir "clase"
y_pred.columns = ["probs"]
both = pd.concat([y_train,y_pred],axis=1) #Juntamos el valor real contra el valor predicho
print(both)

In [ ]:
both[['Clase','probs']]

In [ ]:
#zeros['_tmpkey'] = 1

In [ ]:
zeros = both[['Clase','probs']][both['Clase']==0] # filtro para tomar solo los valores que tienen Clase ==0 #Data_Frame
ones = both[['Clase','probs']][both['Clase']==1] # filtro para tomar solo los valores que tienen Clase ==1 #Data_Frame
def df_crossjoin(df1, df2, **kwargs):
    df1['_tmpkey'] = 1 # Agrega una nueva columna llena de "unos" al df1 
    df2['_tmpkey'] = 1 # Agrega una nueva columna llena de "unos" al df2
    res = pd.merge(df1, df2, on='_tmpkey', **kwargs).drop('_tmpkey',axis=1) # Parecido al comando INNER JOIN "" ON "" de SQL
    # en donde el header de la columna '_tmpkey' se usa como elemento común para unior df1 y df2 en un solo Data Frame y con 
    # drop eliminamos ambas columnas de "_tmpkey" creadas previamente
    
    # Se multplican 209 filas por 491 filas = 102,619 filas 
    
    res.index = pd.MultiIndex.from_product((df1.index, df2.index)) # Hace multi índices a partir del producto cartesiano
    #de múltiples iterables. 
    
    #print(res)
    
    #df1.drop('_tmpkey', axis=1, inplace=True) # no retorna ningún valor 
    #df2.drop('_tmpkey', axis=1, inplace=True) # no retorna ningún valor
    return res

In [ ]:
#ones

In [ ]:
#zeros

In [ ]:
joined_data = df_crossjoin(ones,zeros)

In [ ]:
joined_data

In [ ]:
#joined_data['concordant_pair'] = 0

In [ ]:
joined_data

In [ ]:
#joined_data['discordant_pair']

In [ ]:
#sum(joined_data['discordant_pair'])

In [ ]:
#(sum(joined_data['discordant_pair'])*1.0 )

In [ ]:
(joined_data.shape[0])

In [ ]:
#sum(joined_data['tied_pair'])

In [ ]:
#(sum(joined_data['discordant_pair'])*1.0 )/(joined_data.shape[0])

In [ ]:
 #(sum(joined_data['concordant_pair'])*1.0 )

In [ ]:
(joined_data.shape[0])

In [ ]:
#(sum(joined_data['concordant_pair'])*1.0 )/(joined_data.shape[0])

In [ ]:
 #(sum(joined_data['discordant_pair'])*1.0 )/(joined_data.shape[0])

# Cálculo del estadístico C

In [ ]:
joined_data['concordant_pair'] = 0 # Se agrega una columna llamada "concordant_pair"
joined_data.loc[joined_data['probs_x'] > joined_data['probs_y'], # si probs_x (predicción) > probs_y "concordant_pair"=1
'concordant_pair'] =1                                            # Un par es concordante si la probabilidad contra la clase 1
                                                                 # es mayor que la clase 0 

joined_data['discordant_pair'] = 0
joined_data.loc[joined_data['probs_x'] < joined_data['probs_y'], #discordante si la probabilidad contra la clase 1 es menor 
'discordant_pair'] =1                                            #que la clase 0

joined_data['tied_pair'] = 0                                     # Si ambas probabilidades son iguales, las ponemos 
joined_data.loc[joined_data['probs_x'] ==                        # en la categoría de par empatado.
joined_data['probs_y'],'tied_pair'] =1                           # si probs_x (predicción) == probs_y "tied_pair"=1

p_conc = (sum(joined_data['concordant_pair'])*1.0 )/(joined_data.shape[0]) # suma el total de pares concordantes y lo vuelve 
                                                                           # decimal y se divide entre el total de pares con-
                                                                           # cordantes para ver el porcentaje de pares que son 
                                                                           # concordantes

p_disc = (sum(joined_data['discordant_pair'])*1.0 )/(joined_data.shape[0]) # suma el total de pares disconcordantes y lo vuelve 
                                                                           # decimal y se divide entre el total de pares con-
                                                                           # cordantes para ver el porcentaje de pares que son 
                                                                           # disconcordantes


c_statistic = 0.5 + (p_conc - p_disc)/2.0
print ("\nC-statistic:",round(c_statistic,4))


In [ ]:
both['probs']

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import auc


fpr, tpr, thresholds = metrics.roc_curve(both['Clase'],both['probs'], #Calcula Receiver operating characteristic (ROC).
pos_label=1) # both['Clase']--> Etiquetas binarias verdaderas. both['probs']--> Puntuaciones objetivo 
             # Si y_true está entre {-1, 1} o {0, 1}, pos_label se esablece en 1

roc_auc = auc(fpr,tpr) # Calcule el área bajo la curva (AUC) utilizando la regla trapezoidal. 
plt.figure()           # Coordenadas x --> fpr ; Coordenadas y --> tpr
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='Curva ROC (area =%0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1-Specificity)')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC - German Credit Data')
plt.legend(loc="lower right")
plt.show()

# Variables eliminadas una por una.

In [ ]:
credit_data_new.columns.values

In [ ]:
remove_cols_extra_dummy = ['estado_cuenta_exist_A11', 'hist_cred_A30',
'propósito_A40', 'cuent_ahorros_A61','emp_act_desde_A71','estado_pers_sexo_A91',
'otros_deud_A101','propiedad_A121', 'otros_planes_cuot_A141','trabajador_extranj_A201']

In [ ]:
remove_cols_insig = ['propósito_A46', 'propósito_A45', 'propósito_A44',
'cuent_ahorros_A63','otros_planes_cuot_A143','propiedad_A123',
'estado_cuenta_exist_A12', 'hist_cred_A32', 'hist_cred_A33',
'propósito_A410','propósito_A49', 'propósito_A48',
'propiedad_A122', 'estado_pers_sexo_A92','trabajador_extranj_A202','estado_pers_sexo_A94',
'propósito_A42','otros_deud_A102','Edad_en_años','cuent_ahorros_A64','cuent_ahorros_A62',
'cuent_ahorros_A65','otros_deud_A103','emp_act_desde_A72','emp_act_desde_A73','emp_act_desde_A74','emp_act_desde_A75']

In [ ]:
# Aquí, hemos creado la lista adicional para eliminar variables insignificantes paso a paso de forma iterativa mientras 
# trabajamos en la metodología de eliminación hacia atrás; después del final de cada iteración, seguiremos agregando la 
# variable más insignificante y multicolineal a la lista remove_cols_insig, para que esas variables se eliminen mientras 
# se entrena el modelo.
#remove_cols_insig = []
remove_cols1 = list(set(remove_cols_extra_dummy+remove_cols_insig))

In [ ]:
remove_cols1

## Regresión Logística con nuevas variables

In [ ]:
import statsmodels.api as sm
logistic_model = sm.Logit(y_train, sm.add_constant(x_train.drop(
remove_cols1, axis=1))).fit()
print (logistic_model.summary())

In [ ]:
print ("\nFactor de inflación de varianza(Variance Inflation Factor)")
cnames = x_train.drop(remove_cols1,axis=1).columns #elimina la primera columna de cada categoría en x_train reduciendo el data
#set original de 50 columnas a 40 columnas

for i in np.arange(0,len(cnames)): # se crea un array Numpy que va desde 0 hasta a longitud de cnames (en este caso 40)
    
    xvars = list(cnames) #lista que contiene los nombres de las columnas ya con el filtro de "cnames"
    
    yvar = xvars.pop(i) # los 40 valores de cada columna siendo esttas las variables de respuesta (pop(i)) extrae el elemento
    #i-ésimo de la lista xvars
    
    mod = sm.OLS(x_train.drop(remove_cols1,axis=1)[yvar], sm.add_constant(x_train.drop(remove_cols1,axis=1)[xvars])) #calcula
    # los mínimos cuadrados ordinarios (OLS)
    # model = sm.OLS(Y,X) <---- sintaxis
    
    # X ---> sm.add_constant(x_train.drop(remove_cols,axis=1)[xvars]) ---> 40 data sets cuya primera columna es una columna de 
    # "unos" agregada a cada data set que antes poseía 39 coumnas al serle extraído una columna del data set original
    
    # Se agrega una columna de unos como una variable dependiente. La razón por la que tiene que hacer esto 
    # es porque la función de regresión asume que si no hay una columna constante, entonces desea ejecutar la regresión sin 
    #una intersección. 
    
    # Y ---> x_train.drop(remove_cols,axis=1)[yvar] ---> Cada header de cada columna se usa como variable de respuesta
    
    res = mod.fit() # Se calculan 40 regresiones con mínimos cuadrados
    vif = 1/(1-res.rsquared)
    print (yvar,round(vif,3))

In [ ]:
y_pred1 = pd.DataFrame(logistic_model.predict(sm.add_constant(x_train.drop (remove_cols1,axis=1)))) #Usamos e conjunto de prueba 
# para prededcir el valor de la variable de respuesta o a predecir "clase"
y_pred1.columns = ["probs"]
both = pd.concat([y_train,y_pred1],axis=1) #Juntamos el valor real contra el valor predicho
print(both)

In [ ]:
zeros = both[['Clase','probs']][both['Clase']==0] # filtro para tomar solo los valores que tienen Clase ==0 #Data_Frame
ones = both[['Clase','probs']][both['Clase']==1] # filtro para tomar solo los valores que tienen Clase ==1 #Data_Frame
def df_crossjoin(df1, df2, **kwargs):
    df1['_tmpkey'] = 1 # Agrega una nueva columna llena de "unos" al df1 
    df2['_tmpkey'] = 1 # Agrega una nueva columna llena de "unos" al df2
    res = pd.merge(df1, df2, on='_tmpkey', **kwargs).drop('_tmpkey',axis=1) # Parecido al comando INNER JOIN "" ON "" de SQL
    # en donde el header de la columna '_tmpkey' se usa como elemento común para unior df1 y df2 en un solo Data Frame y con 
    # drop eliminamos ambas columnas de "_tmpkey" creadas previamente
    
    # Se multplican 209 filas por 491 filas = 102,619 filas 
    
    res.index = pd.MultiIndex.from_product((df1.index, df2.index)) # Hace multi índices a partir del producto cartesiano
    #de múltiples iterables. 
    
    #print(res)
    
    #df1.drop('_tmpkey', axis=1, inplace=True) # no retorna ningún valor 
    #df2.drop('_tmpkey', axis=1, inplace=True) # no retorna ningún valor
    return res

In [ ]:
joined_data1 = df_crossjoin(ones,zeros)

In [ ]:
joined_data1

In [ ]:
joined_data1['concordant_pair'] = 0 # Se agrega una columna llamada "concordant_pair"
joined_data1.loc[joined_data1['probs_x'] > joined_data1['probs_y'], # si probs_x (predicción) > probs_y "concordant_pair"=1
'concordant_pair'] =1                                            # Un par es concordante si la probabilidad contra la clase 1
                                                                 # es mayor que la clase 0 

joined_data1['discordant_pair'] = 0
joined_data1.loc[joined_data1['probs_x'] < joined_data1['probs_y'], #discordante si la probabilidad contra la clase 1 es menor 
'discordant_pair'] =1                                            #que la clase 0

joined_data1['tied_pair'] = 0                                     # Si ambas probabilidades son iguales, las ponemos 
joined_data1.loc[joined_data1['probs_x'] ==                        # en la categoría de par empatado.
joined_data1['probs_y'],'tied_pair'] =1                           # si probs_x (predicción) == probs_y "tied_pair"=1

p_conc = (sum(joined_data1['concordant_pair'])*1.0 )/(joined_data1.shape[0]) # suma el total de pares concordantes y lo vuelve 
                                                                           # decimal y se divide entre el total de pares con-
                                                                           # cordantes para ver el porcentaje de pares que son 
                                                                           # concordantes

p_disc = (sum(joined_data1['discordant_pair'])*1.0 )/(joined_data1.shape[0]) # suma el total de pares disconcordantes y lo vuelve 
                                                                           # decimal y se divide entre el total de pares con-
                                                                           # cordantes para ver el porcentaje de pares que son 
                                                                           # disconcordantes


c_statistic = 0.5 + (p_conc - p_disc)/2.0
print ("\nC-statistic:",round(c_statistic,4))


In [ ]:
joined_data1

In [ ]:
joined_data

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import auc


fpr, tpr, thresholds = metrics.roc_curve(both['Clase'],both['probs'],
pos_label=1)
roc_auc = auc(fpr,tpr)
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='Curva ROC (area =%0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1-Specificity)')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC - German Credit Data')
plt.legend(loc="lower right")
plt.show()

# Falta analizar el código de abajo

In [ ]:
both

In [ ]:
both["y_pred"] = 0

In [ ]:
both

In [ ]:
both.loc[both["probs"]>0.1,'y_pred']

In [ ]:
accuracy_score

In [ ]:
# Una vez que hemos encontrado la mejor situación del conjunto de datos de entrenamiento, la siguiente y última tarea 
# es predecir la categoría desde la probabilidad hasta el valor predeterminado. Hay muchas formas de establecer el valor de 
# umbral para convertir la probabilidad predicha en una clase real. En el siguiente código, hemos realizado una búsqueda de 
# cuadrícula simple para determinar el límite de umbral de mejor probabilidad. No obstante, incluso las curvas de sensibilidad 
# y especificidad podrían utilizarse para esta tarea.



for i in list(np.arange(0,1,0.1)): # crear array de 0 a 1 en pasos de 0.1
    both["y_pred"] = 0 # agregar columna "y_pred" con valores cero
    both.loc[both["probs"] > i, 'y_pred'] = 1 # Si e lvalor de la columna probs es mayor que los vcalores i
                                            # se le asigna valor de 1 
    print ("Threshold",i,"Train Accuracy:",
round(accuracy_score(both['Clase'], both['y_pred']),4)) # Se asigna la precisión 

# accuracy_score(y_true, y_pred)

In [ ]:
# Ahora, se aplicará un umbral de 0,5 a los datos de prueba para verificar si el modelo es consistente en varios conjuntos
# de datos con el siguiente código:
both["y_pred"] = 0
both.loc[both["probs"] > 0.5, 'y_pred'] = 1
print ("\nTrain Confusion Matrix\n\n", pd.crosstab(both['Clase'],
both['y_pred'],rownames = ["Actuall"],colnames = ["Predicted"]))

print ("\nTrain Accuracy:",round(accuracy_score(both['Clase'],both['y_pred']),4))

In [ ]:
y_pred_test = pd.DataFrame( logistic_model.predict(sm.add_constant(x_test.drop(remove_cols1,axis=1)))) #Data frame con predicciones
y_pred_test.columns = ["probs"] # Se le asigna el nombre "probs" a la columna de predicciones

both_test = pd.concat([y_test,y_pred_test],axis=1) # se unen los daros reales y predichos del conjunto de prueba en un sollo DataFrame

both_test["y_pred"] = 0 # Se asigna una columna con ceros (300 ceros en total)}

both_test.loc[both_test["probs"] > 0.5, 'y_pred'] = 1 # Se filtran datos para asignar 0 o 1 según el umbral 0.5

print ("\nPrueba de la Matriz de Confusión\n\n", pd.crosstab( both_test['Clase'],
both_test['y_pred'],rownames = ["Real"],colnames = ["Predicho"]))

print ("\nPrecisión de la prueba(Test Accuracy):", round(accuracy_score( both_test['Clase'],
both_test['y_pred']),4))